<a href="https://colab.research.google.com/github/fezjo/ml-project/blob/master/ml_project_ciz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/fezjo/ml-project/raw/master/chat_history.zip
!unzip -o -Pfmfilords chat_history.zip
!pip install unidecode

--2023-01-31 11:50:10--  https://github.com/fezjo/ml-project/raw/master/chat_history.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fezjo/ml-project/master/chat_history.zip [following]
--2023-01-31 11:50:10--  https://raw.githubusercontent.com/fezjo/ml-project/master/chat_history.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2804699 (2.7M) [application/zip]
Saving to: ‘chat_history.zip.6’

chat_history.zip.6  100%[===================>]   2.67M  --.-KB/s    in 0.07s   

2023-01-31 11:50:10 (40.7 MB/s) - ‘chat_history.zip.6’ saved [2804699/2804699]

Archive:  chat_history.zip
  inflatin

In [2]:
from matplotlib import pyplot as plt

def plot_history(history):
  dpi = 100
  plt.figure(dpi=dpi)
  plt.plot(history.history['loss'], label='training loss')
  plt.plot(history.history.get('val_loss', []), label='validation loss')
  plt.legend(loc='best')

  plt.figure(dpi=dpi)
  plt.plot(history.history['accuracy'], label='train accuracy')
  plt.plot(history.history.get('val_accuracy', []), label='validation accuracy')
  plt.legend(loc='best')
  plt.show()

In [3]:
import json

with open("chat_history.json") as chf:
  data = json.load(chf)

In [4]:
from unidecode import unidecode

author_blacklist = {"PollBot", "Dartboi", "Telelog"}
authors = dict()
messages = []
for message in data["messages"]:
  if message["type"] != "message":
    continue
  text_entities, author = message["text_entities"], message["from"]
  if not text_entities or author in author_blacklist:
    continue
  text_chunks = []
  for entity in text_entities:
    text_chunks.append(entity["text"])
  text = "".join(text_chunks)
  text = unidecode(text).lower()
  authors.setdefault(author, len(authors))
  messages.append([text, authors[author]])

print(authors)
print(messages[:100])

{'Jozef': 0, 'Bohdanator': 1, 'Daniel Oravec': 2, 'Samuel Čavoj': 3, 'Dávid Mišiak': 4, 'Pavol Kebis': 5, 'Jaroslav Paška': 6, 'Ján Priner': 7}
[['make me admin pls', 0], ['sure :d :d', 1], ['aj mna!!', 2], ['aj mna!!!', 3], ['', 3], ['wtf', 1], ['ono to islo dokelu tak rychlo...', 1], ['neviem, co si cakal, ze sa stane, ked z jozefa spravis admina', 3], ['nespravil som :d :d', 1], ['jaj', 3], ['zabudol som nastavit aby sa to nedialo', 1], [':(', 2], ['genialne', 3], ['kde su vsetky prvacky?', 0], ['co tu robi paska?', 3], ['flexi', 0], ['ani nie', 3], ['tu neexistuje demokracia', 1], ['preco ste dnes neboli v skole?', 2], ['just kidding', 1], ['podme sa bavit o kamenoch', 1], ['lebo som bol na orave', 1], ['rebels', 4], ['pridte na vba231', 0], ['?', 4], ['kedy sa zacina skola?', 0], ['vyzera to tak ze v pondelok', 4], ['cas?', 0], ['asi podla rozvrhu', 4], ['alebo to mecheche otvorenie akademickeho roka', 4], ['neviem ktore z toho', 4], ['vie niekto?', 4], ['podla mna normalne podla 

In [5]:
author_activity = [0] * len(authors)
for message in messages:
  author_activity[message[1]] += 1

for author, activity in zip(authors.items(), author_activity):
  print("{}: {} [{:.2%}]".format(author, activity, activity / len(messages)))

('Jozef', 0): 17808 [27.98%]
('Bohdanator', 1): 6326 [9.94%]
('Daniel Oravec', 2): 6078 [9.55%]
('Samuel Čavoj', 3): 17254 [27.11%]
('Dávid Mišiak', 4): 15336 [24.10%]
('Pavol Kebis', 5): 489 [0.77%]
('Jaroslav Paška', 6): 21 [0.03%]
('Ján Priner', 7): 328 [0.52%]


In [6]:
import numpy as np
message_lengths = np.array(list(map(lambda m: len(m[0]), messages)))
sml = sorted(message_lengths)
ml_avg, ml_std = np.mean(message_lengths), np.std(message_lengths)
print(f"message length / avg: {ml_avg:.2f} / std: {ml_std:.2f}")
for r in (0.5, 0.68, 0.9, 0.95, 0.99, 1):
    i = min(len(sml) - 1, int(len(sml) * r))
    print(r, sml[i])

message length / avg: 34.47 / std: 60.23
0.5 23
0.68 35
0.9 72
0.95 98
0.99 189
1 4095


In [7]:
shuffled_messages = np.random.permutation(messages)
X, Y = map(lambda k: np.array([m[k] for m in shuffled_messages]), (0, 1))
train_test_ratio = 0.8
num_train = int(train_test_ratio * len(X))
X_train, X_test = np.split(X, [num_train])
Y_train, Y_test = np.split(Y, [num_train])
output_shape = len(authors)

print(X[:100])
print(Y[:100])
print(X.shape, X_train.shape, X_test.shape)
print(Y.shape, Y_train.shape, Y_test.shape)

['mne to trvalo asi pol hodiny'
 'este si tie prednasky mozes pustat na loope pocas spanku' 'kul'
 'ja spravim ksp mimo tyzdna' 'aha' 'no ani na to nemali bohov :d' '3?'
 'brutalne som to zoptimalizoval prave' 'ako to ze este nezacala fpro?'
 'nula celych nula celych nula nula'
 'oukej  lebo my sme sa ho pytali, kedze ostatni to maju urcene, a teraz odpisal toto'
 'no akoze asi je to take, ze ak to najdes, tak si vyhral' 'praca!'
 'david bud prosim aktivny'
 'na pisomke napis ku kazdej ulohe ze "riesenie existuje"'
 'proste to budem riesit zajtra a ked to nevyriesim, tak nestratim 20 hodin svojho casu'
 'odpisala pardubska dnes rano, ze:\n"dakujem"'
 'asi zlozitejsi nejaky pre nieco komplexnejsie'
 'mne na konci skusky povedal nieco take ze "dovidenia, hadam sa este uvidime na nejakom dalsom predmete", takze neviem-neviem'
 'ked som testoval' 'viac menej vsetko co napise eli je dobry writeup'
 'fakin 1000 sprav' 'take nieco' 'este pts na natrhne asi' 'velmi malo'
 'najdi jej nadkluce' 

In [21]:
import tensorflow as tf
from tensorflow import keras


# Use the TextVectorization layer to tokenize the input string based on bigrams
bigram_sequence_dim = int(ml_avg + 2 * ml_std)
preprocessing = keras.layers.TextVectorization(
    standardize=None,
    split=tf.strings.bytes_split,
    ngrams=2,
    output_sequence_length=bigram_sequence_dim)
preprocessing.adapt(X_train)

# Input layer for string data
inputs = keras.layers.Input(shape=(1,), dtype=tf.string)

# Preprocessing layer
layer = preprocessing(inputs)

# Embedding layer
max_vocab_size = len(preprocessing.get_vocabulary()) + 1
print(f"Vocabulary size: {max_vocab_size}")
embedding_dim = 200
layer = keras.layers.Embedding(
    input_dim=max_vocab_size,
    output_dim=embedding_dim,
#     embeddings_regularizer="l1",
)(layer)

# Add a dropout layer to prevent overfitting
layer = keras.layers.Dropout(rate=0.25)(layer)

# Add dimension for channel needed by convolution
layer = keras.layers.Reshape(target_shape=(*layer.shape[-2:], 1))(layer)

# # Create 3 parallel branches for convolution with varying window widths
n_conv_filters = 300 # TODO
parent_layer = layer
child_layers = []
for ks in (3, 4, 5):
    layer = parent_layer

    layer = keras.layers.Conv2D(
        filters=n_conv_filters,
        kernel_size=(ks, embedding_dim),
        activation="selu")(layer)
    
    # Max pooling layer
    layer = keras.layers.GlobalMaxPooling2D()(layer)
    child_layers.append(layer)

# Concatenate the outputs of the parallel Conv1D layers
layer = keras.layers.Concatenate()(child_layers)

# # Add a dropout layer to prevent overfitting
layer = keras.layers.Dropout(rate=0.25)(layer)

# # Add a dense layer
# layer = keras.layers.Dense(units=300, activation="relu")(layer)

# Output layer
outputs = keras.layers.Dense(units=output_shape, activation="softmax")(layer)

# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"])
model.summary()

Vocabulary size: 3458
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
text_vectorization_2 (TextVecto (None, 154)          0           input_3[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 154, 200)     691600      text_vectorization_2[0][0]       
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 154, 200)     0           embedding_2[0][0]                
_____________________________________________________________________

In [9]:
print(preprocessing.get_vocabulary()[:100])

['', '[UNK]', ' ', 'a', 'e', 'o', 't', 'i', 'n', 's', 'm', 'l', 'r', 'd', 'k', 'e  ', 'p', 'v', 'u', 'c', 'z', 'o  ', 'a  ', '  t', 'j', '  s', 'y', 'b', 'h', '  n', 't o', 'n e', '  p', 'm  ', '  a', 'i  ', 'a k', 'a l', '  m', 'n a', 't a', 'i e', 's t', '  z', 't e', 'p r', '  v', 's i', ',', 'y  ', 'l e', ',  ', 'o m', 'n i', 'p o', 'r a', 'm a', 'j e', 'r e', '?', 'a t', 'e d', 'k o', 't  ', 'f', 'o v', '  d', '  c', '  j', 'u  ', 'e n', 'l  ', 'z e', 'a m', '  b', 'a s', 'e s', 's a', 'r o', 'c h', 'i n', 'e m', 'd a', 'd e', 'a n', 's  ', 'c i', 'k  ', '  k', 'l a', 'v e', 'v i', 's o', 'e r', '.', 't i', 'v a', 'n o', 'i s', 'r i']


In [17]:
sample_text = "fakin ak si nic nedas, tak PON UTO PIA mas???!? cely den na pracu a este streda poobede :D 👍 @fezjo"

inp = model.input                                           # input placeholder
model_outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = lambda outputs: keras.backend.function(inp, outputs)   # evaluation function

# Testing
# for output in model_outputs:
#   print(functor([output])(np.array([sample_text])))

In [ ]:
history = model.fit(**{
    "x": X_train,
    "y": keras.utils.to_categorical(Y_train),
    "epochs": 20,
    "batch_size": 512,
    "validation_split": 0.10,
    "verbose": 1,
    })

plot_history(history)

Epoch 1/20
90/90 [==============================] - 10s 111ms/step - loss: 1.1972 - accuracy: 0.5368 - val_loss: 1.4259 - val_accuracy: 0.4399
Epoch 2/20
90/90 [==============================] - 10s 110ms/step - loss: 1.1769 - accuracy: 0.5484 - val_loss: 1.4334 - val_accuracy: 0.4352
Epoch 3/20
90/90 [==============================] - 10s 111ms/step - loss: 1.1576 - accuracy: 0.5570 - val_loss: 1.4340 - val_accuracy: 0.4423
Epoch 4/20
90/90 [==============================] - 10s 111ms/step - loss: 1.1381 - accuracy: 0.5656 - val_loss: 1.4363 - val_accuracy: 0.4395
Epoch 5/20
90/90 [==============================] - 10s 110ms/step - loss: 1.1177 - accuracy: 0.5734 - val_loss: 1.4512 - val_accuracy: 0.4375
Epoch 6/20
90/90 [==============================] - 10s 110ms/step - loss: 1.1032 - accuracy: 0.5800 - val_loss: 1.4475 - val_accuracy: 0.4413
Epoch 7/20
90/90 [==============================] - 10s 111ms/step - loss: 1.0844 - accuracy: 0.5895 - val_loss: 1.4647 - val_accuracy: 0.4417

In [12]:
train_score = model.evaluate(X_train, tf.keras.utils.to_categorical(Y_train))
print("\n\ntrain loss: {} | train acc: {}\n".format(train_score[0], train_score[1]))

test_score = model.evaluate(X_test, tf.keras.utils.to_categorical(Y_test))
print("\n\ntest loss: {} | test acc: {}".format(test_score[0], test_score[1]))

 573/1591 [=========>....................] - ETA: 5s - loss: 0.7606 - accuracy: 0.7264

KeyboardInterrupt: 

```
Epoch 1/10
716/716 [==============================] - 27s 36ms/step - loss: 1.4964 - accuracy: 0.3837 - val_loss: 1.4383 - val_accuracy: 0.4183
Epoch 2/10
716/716 [==============================] - 25s 36ms/step - loss: 1.4131 - accuracy: 0.4310 - val_loss: 1.3970 - val_accuracy: 0.4354
Epoch 3/10
716/716 [==============================] - 25s 36ms/step - loss: 1.3427 - accuracy: 0.4655 - val_loss: 1.3826 - val_accuracy: 0.4470
Epoch 4/10
716/716 [==============================] - 25s 36ms/step - loss: 1.2733 - accuracy: 0.4990 - val_loss: 1.3885 - val_accuracy: 0.4440
Epoch 5/10
716/716 [==============================] - 25s 35ms/step - loss: 1.1959 - accuracy: 0.5339 - val_loss: 1.3993 - val_accuracy: 0.4521
Epoch 6/10
716/716 [==============================] - 25s 35ms/step - loss: 1.1183 - accuracy: 0.5708 - val_loss: 1.4258 - val_accuracy: 0.4544
Epoch 7/10
716/716 [==============================] - 25s 35ms/step - loss: 1.0404 - accuracy: 0.5998 - val_loss: 1.4407 - val_accuracy: 0.4487
Epoch 8/10
716/716 [==============================] - 25s 35ms/step - loss: 0.9775 - accuracy: 0.6283 - val_loss: 1.4890 - val_accuracy: 0.4476
Epoch 9/10
716/716 [==============================] - 25s 35ms/step - loss: 0.9131 - accuracy: 0.6558 - val_loss: 1.5166 - val_accuracy: 0.4470
Epoch 10/10
716/716 [==============================] - 25s 35ms/step - loss: 0.8509 - accuracy: 0.6859 - val_loss: 1.5764 - val_accuracy: 0.4550
```